In [50]:
import numpy as np
from numpy import linalg as la

The following is the system of ODEs that we will use:
<br>
$\frac{dS}{Dt} = -\frac{S}{N}[\beta_pI_p + \beta_aI_a + \beta_sI_s]$
<br>
$\frac{dE}{Dt} = \frac{S}{N}[\beta_pI_p + \beta_aI_a + \beta_sI_s] - \frac{1}{\tau_E}$
<br>
$\frac{dI_p}{dt} = \frac{1}{\tau_E}E - \frac{1}{\tau_P}I_p$
<br>
$\frac{dI_a}{dt} = \frac{f}{\tau_P}I_p - \frac{1}{\tau_I}I_a$
<br>
$\frac{dI_s}{dt} = \frac{1 - f}{\tau_P}I_p - \frac{1 - d}{\tau_I}I_s - \frac{d}{\tau_D}I_s$
<br>
$\frac{dR}{dt} = \frac{1}{\tau_I}I_a + \frac{1 - d}{\tau_I}I_s$

Where: 
<br>
$S$: Susceptible Population
<br>
$N$: Total Population
<br>
$E$: Exposed Population
<br>
$I_p$: Infected Population (pre-symptomatic)
<br>
$I_a$: Infected Population (asymptomatic)
<br>
$I_s$: Infected Population (symptomatic)
<br>
$R$: Recovered Population
<br><br>
$\beta_p$: Infection Rate (pre-symptomatic)
<br>
$\beta_a$: Infection Rate (asymptomatic)
<br>
$\beta_s$: Infection Rate (symptomatic)
<br><br>
$\tau_E$: Length of time exposed
<br>
$\tau_P$: Length of time pre-infectious
<br>
$\tau_I$: Length of time infectious
<br>
$\tau_D$: Length of time while patient is dying (...can't think of a less morbid way to describe...)
<br><br>
$f$: Proportion of infected people who are asymptomatic
<br>
$d$: Proportion of infected people who die

In [51]:
V = np.array([[1/tauE,0,0,0],[-1/tauE,1/tauP,0,0],[0,-f/tauP,1/tauI,0],[0,-(1-f)/tauP,0,(1-d)/tauI + d/tauD]])

In [52]:
F = np.array([[0,beta,beta,beta],[0,0,0,0],[0,0,0,0],[0,0,0,0]])

In [53]:
V

array([[ 0.28571429,  0.        ,  0.        ,  0.        ],
       [-0.28571429,  0.66666667,  0.        ,  0.        ],
       [ 0.        , -0.2       ,  0.28571429,  0.        ],
       [ 0.        , -0.46666667,  0.        ,  0.275     ]])

In [58]:
F*la.inv(V)

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [54]:
F

array([[0. , 0.8, 0.8, 0.8],
       [0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. ]])

Reproductive Rate: $R_0 = FV^-1$

In [56]:
R_0 = np.max(la.eig(F*la.inv(V)))

ValueError: could not broadcast input array from shape (4,4) into shape (4)

In [9]:
def covid(y, t, betaP, betaA, betaS, tauE, tauP, tauI, tauD, f, d, N):
    S, E, Ip, Ia, Is, R = y
    dydt = [-(S/N)*(betaP*Ip + betaA*Ia + betaS*Is), (S/N)*(betaP*Ip + betaA*Ia + betaS*Is) - (1/tauE)*E, (1/tauE)*E - (1/tauP)*Ip, (f/tauP)*Ip - (1/tauI)*Ia, ((1-f)/tauP)*Ip - ((1-d)/tauI)*Is - (d/tauD)*Is, (1/tauI)*Ia + ((1-d)/tauI)*Is]
    return dydt

In [16]:
def CalculateR_0(tauE,tauP,tauI,tauD,f,d,beta,N):
##Function for calculating the basic reproductive rate for a given set of parameters##

    V = zeros(4,4);
    F = zeros(4,4);

##Input non-zero entries of F##
F(1,[2,3,4]) = beta;

##Input non-zero entries of V##
V(1,1) = 1/tauE;
V(2,1) = -1/tauE;
V(2,2) = 1/tauP;
V(3,2) = -f/tauP;
V(3,3) = 1/tauI;
V(4,2) = -(1-f)/tauP;
V(4,4) = (1-d)/tauI + d/tauD;

y = max(eigs(F*inv(V)));

end


SyntaxError: cannot assign to function call (<ipython-input-16-978272396f59>, line 8)

$S: x(1)$
<br>
$E: x(2)$
<br>
$I_p: x(3)$
<br>
$I_a: x(4)$
<br>
$I_s: x(5)$

In [59]:
def SEIR(t,x,res,tauE,tauP,tauI,tauD,f,d,beta0,N):
##Equations for SEIR model, with multiple infected classes##
    y(1) = -beta(t,beta0,res)*x(1)*(x(3)+x(4)+x(5))/N;
    y(2) = beta(t,beta0,res)*x(1)*(x(3)+x(4)+x(5))/N-1/tauE*x(2);
    y(3) = 1/tauE*x(2) - 1/tauP*x(3);
    y(4) = f/tauP*x(3) - 1/tauI*x(4);
    y(5) = (1-f)/tauP*x(3) - (1-d)/tauI*x(5) - d/tauD*x(5);
    y(6) = 1/tauI*x(4) + (1-d)/tauI*x(5);

SyntaxError: cannot assign to function call (<ipython-input-59-fa941bd19e7f>, line 3)

In [10]:
betaP = 0.8
betaA = 0.8
betaS = 0.8
tauE = 3.5
tauP = 1.5
tauI = 3.5
tauD = 14
f = 0.3
d = 0.05
N = 4.9*10**6

In [61]:
def SEIR_equations(t,x,res,tauE,tauP,tauI,tauD,f,d,beta0,N):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-61-7525c0f91a12>, line 1)

In [63]:
def y1(beta,t,beat0,res,N):
    -beta(t,beta0,res)*x(1)*(x(3)+x(4)+x(5))/N

In [64]:
y1(0.8,70,)

<function __main__.y1(beta, t, beat0, res, N)>

In [3]:
def covid(y, t, betaP, betaA, betaS, tauE, tauP, tauI, tauD, f, d, N):
    S, E, Ip, Ia, Is, R = y
    dydt = [-(S/N)*(betaP*Ip + betaA*Ia + betaS*Is), (S/N)*(betaP*Ip + betaA*Ia + betaS*Is) - (1/tauE)*E, (1/tauE)*E - (1/tauP)*Ip, (f/tauP)*Ip - (1/tauI)*Ia, ((1-f)/tauP)*Ip - ((1-d)/tauI)*Is - (d/tauD)*Is, (1/tauI)*Ia + ((1-d)/tauI)*Is]
    return dydt

In [65]:
res

NameError: name 'res' is not defined

In [2]:
#model parameters
tauE = 3.5; 
tauP = 1.5;
tauI = 3.5;
tauD = 14;
f = 0.3;
d = 0.05;
beta = 0.8;
N = 4.9*10^6;
t_offset = 7;
#much better fit with t_offset = 9

TypeError: unsupported operand type(s) for ^: 'float' and 'int'